In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

loader = TextLoader("files/love-story.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

# The Neo4jVector Module will connect to Neo4j and create a vector index if needed.

URL = os.getenv("URL")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

db = Neo4jVector.from_documents(
    docs, OpenAIEmbeddings(), url=URL, username=USERNAME, password=PASSWORD
)

query = "who is Harry"
docs_with_score = db.similarity_search_with_score(query, k=2)

Created a chunk of size 252, which is longer than the specified 10
Created a chunk of size 239, which is longer than the specified 10
Created a chunk of size 234, which is longer than the specified 10
Created a chunk of size 285, which is longer than the specified 10
Created a chunk of size 253, which is longer than the specified 10
Created a chunk of size 199, which is longer than the specified 10
Created a chunk of size 264, which is longer than the specified 10
Created a chunk of size 249, which is longer than the specified 10


In [3]:
docs_with_score

[(Document(page_content='Harry was a shy, unassuming guy who worked at a bookstore in a small college town. He spent his days surrounded by dusty tomes, reorganizing the shelves and ringing up purchases, occasionally stealing glances at the attractive students who wandered in.', metadata={'source': 'files/love-story.txt'}),
  0.8912085294723511),
 (Document(page_content='Harry had gone from a shy introvert to a man brimming with love, confidence and purpose. And Sophia was the one who had unlocked that potential within him through her kindness, patience and adoration. Together, they would go forth into the world, bound by an unbreakable bond and endless future.', metadata={'source': 'files/love-story.txt'}),
  0.8800807595252991)]

In [8]:
from neo4j import GraphDatabase

# Connect to the Neo4j database
URL = os.getenv("URL")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

driver = GraphDatabase.driver(URL, auth=(USERNAME, PASSWORD))


# Function to execute a write query
def write_query(tx, query):
    tx.run(query)

# Cypher queries
queries = [
    "CREATE (harry:Person {name: 'Harry'})",
    "CREATE (sophia:Person {name: 'Sophia'})",
    "CREATE (bookstore:Place {name: 'Bookstore'})",
    "CREATE (collegeTown:Place {name: 'College Town'})",
    "CREATE (park:Place {name: 'Park'})",
    "CREATE (coffeeShop:Place {name: 'Coffee Shop'})",
    "CREATE (creek:Place {name: 'Creek'})",
    "CREATE (cityOverlook:Place {name: 'City Overlook'})",
    "CREATE (wedding:Event {name: 'Wedding'})",
    "MERGE (harry)-[:WORKS_AT]->(bookstore)",
    "MERGE (harry)-[:LIVES_IN]->(collegeTown)",
    "MERGE (sophia)-[:VISITS]->(bookstore)",
    "MERGE (harry)-[:SMITTEN_WITH]->(sophia)",
    "MERGE (harry)-[:OFFERS_UMBRELLA_TO]->(sophia)",
    "MERGE (harry)-[:WALKS_HOME_WITH]->(sophia)",
    "MERGE (harry)-[:IN_RELATIONSHIP_WITH]->(sophia)",
    "MERGE (harry)-[:TAKES_CLASSES]->(collegeTown)",
    "MERGE (harry)-[:PROPOSES_TO]->(sophia)",
    "MERGE (harry)-[:MARRIES]->(sophia)",
    "MERGE (harry)-[:GO_TO]->(park)",
    "MERGE (harry)-[:GO_TO]->(coffeeShop)",
    "MERGE (harry)-[:GO_TO]->(creek)",
    "MERGE (wedding)-[:CELEBRATES_UNION_OF]->(harry)",
    "MERGE (wedding)-[:CELEBRATES_UNION_OF]->(sophia)"
]

# Execute the queries
with driver.session() as session:
    for query in queries:
        session.write_transaction(write_query, query)

# Close the connection
driver.close()

/tmp/ipykernel_110460/1615192223.py:46: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(write_query, query)
Unable to retrieve routing information
Transaction failed and will be retried in 0.8056052816521065s (Unable to retrieve routing information)
